# Algoritmo de separação de equações diferenciais parciais

## Resumo

Equações diferenciais parciais (EDP) são fundamentais nos campos das ciências e engenharias, principalmente na modelagem matemática de problemas complexos como difusão de calor, propagação de ondas, escoamento de fluidos, crescimento de populações e pertubações do espaço por buracos negros. No entanto, resolver uma EDP analiticamente pode ser muito difícil ou até mesmo impossível, por conta disso foram desenvolvidos diversos métodos de resolução de EDP como separação de variáveis, método das características, transformada Integral, Mudança de variáveis. Apesar de sua centralidade em diversos campos, não existe uma implementação eficiente no software livre SageMath e nos sistemas que o compõem como Sympy. Dada a importância dessa classe de equações, propomos um modulo computacional para o SageMath a fim de automatizar a técnica de separações de variáveis aplicada a EDP, uma implementação modular como essa poderá ser integrada com outras ferramentas de análise ou métodos numéricos, assim contribuindo para o ensino e a pesquisa de EDP.

## Introdução

Equações diferenciais parciais(EDP) são equações envolvendo funções de variais variáveis e suas derivadas parciais, são essenciais na modelagem matemática de de sistemas físicos como propagação de ondas, difusão de ondas e relatividade geral, neste trabalho propomos um modulo computacional para o software livre SageMath a fim de automatizar solução de EDP por meio do método de separação de variáveis.

### Equação diferencial parcial (EDP)

$$A(x,y)\frac{\partial^2 u}{\partial x^2} + B(x,y)\frac{\partial^2 u}{\partial y^2} + C(x,y)\frac{\partial^2 u}{\partial x \partial y} + D(x,y)\frac{\partial u}{\partial x} + E(x,y)\frac{\partial u}{\partial y} + F(x,y)u = G(x,y)$$ 

O método da separação de variáveis consiste em supor que a solução procurada pode ser expressa como produto de funções que dependam de apenas uma variável. Por simplicidade, o método será ilustrado com uma EDP de duas variáveis na equação geral acima, embora o algoritmo e o método possa ser aplicado a equações com número maior de variáveis.

Supomos que a função $u(x,y)$ possa ser escrita como:

\begin{equation}
    u(x,y) = X(x)Y(y)
\end{equation}

Após substituir a suposição acima na equação original, divide-se a Equação por $X(x)Y(y)$ e efetuam-se as manipulações necessárias a fim de dividir a equação em blocos de termos que dependem de apenas uma variável. Caso a suposição seja verdadeira, reescreve-se a EDP da seguinte forma, no contexto de EDP de até segunda ordem:

\begin{equation}
     B_1(X''(x), X'(x), X(x), x) = B_2(Y''(y), Y'(y), Y(y), y)
\end{equation}

Como cada lado da equação depende de uma variável diferente, ambos devem ser igual a uma constante $B_1 =  B_2 = \lambda$. Dessa forma obtemos duas equações diferenciais ordinárias,

\begin{align}
    \begin{cases}
B_1(X''(x), X'(x), X(x), x) = & \lambda\\
B_2(Y''(y), Y'(y), Y(y), y) = & \lambda.
    \end{cases}
\end{align}

Que, em geral, são mais fáceis de se resolver numérica ou analiticamente.

### Exemplo

In [1]:
var('x,y,γ')
F = function('F')(x, y)
X,Y = function('X')(x), function('Y')(y)

**Equação de laplace potencial eletrostatico**

In [2]:
eq = diff(F, x, 2) + diff(F, y, 2) == 0
show(eq)

diff(F(x, y), x, x) + diff(F(x, y), y, y) == 0

In [3]:
eq1 = eq.substitute_function(F == X*Y)
show(eq1)

Y(y)*diff(X(x), x, x) + X(x)*diff(Y(y), y, y) == 0

In [4]:
eq2 = (eq1 / (X*Y)).expand()
show(eq2)

diff(X(x), x, x)/X(x) + diff(Y(y), y, y)/Y(y) == 0

In [5]:
eq3 = eq2 - diff(Y, y, 2)/Y
show(eq3)

diff(X(x), x, x)/X(x) == -diff(Y(y), y, y)/Y(y)

In [6]:
eqx = eq3.lhs() == γ
eqy = eq3.rhs() == γ
show(eqx)
print()
show(eqy)

diff(X(x), x, x)/X(x) == γ

-diff(Y(y), y, y)/Y(y) == γ

## Algoritmo

O algoritmo é composto por várias etapas, mas a separação da EDP ocorre principalmente em nestas três etapas:

**Separável por soma**: Testa se a EDP pode ser expressa por $J(x) + K(y)$, caso essa suposição seja verdade basta diferenciar e após integrar a EDP nas suas respectivas variáveis com o objetivo de encontrar, $J$ e $K$.

**Procura fator comum**: Procura por termos comuns na EDP a fim de evidência-los e por fim simplificar a expressão ou se possível separar a EDP.

**Procura fator comum em termos mistos**: Procura por fator comuns somente nos termos mistos, caso haja um fator comum, é verificado se dividir a EDP como um todo por esse fator não produz novos termos mistos.

``` mermaid

flowchart TD

inicio((Inicio))

partial_separate[Divide EDP pelo produto das componentes]

is_trivially_separable{Trivialmente separável?}

separate_equations[Separa variáveis]

is_sum_separable{Separável por soma?}

sum_separate[Separa EDP]

common_factor1{Fator comum?}

common_factor2{Fator comum?}

partial_common_factor["Procura fator comum em termos mistos"]

recursive["Chama função recursivamente"]

fim1((Fim))
fim2((Fim))

eliminate_common_factor[Procura fator comum]

divide[Divide EDP por fator comum]

return_partially_separated_pde[Retorna EDP parcialmente separada]

inicio --> partial_separate
partial_separate --> is_trivially_separable

is_trivially_separable --> separate_equations
separate_equations --> fim1

is_trivially_separable --> is_sum_separable

is_sum_separable --> sum_separate
sum_separate-->recursive

is_sum_separable --> eliminate_common_factor

eliminate_common_factor --> common_factor1 

common_factor1 --> divide --> recursive
common_factor1 --> partial_common_factor
partial_common_factor --> common_factor2

common_factor2 --> return_partially_separated_pde --> fim2
common_factor2 --> divide 


```



## Modulo de_tools

In [7]:
from de_tools_copy2 import separate_pde # substituir por de_tools para código original
eqt = eq1
eqs = separate_pde(pde=eqt,
                 funcs=(X,Y),
                 sep_variables=(x,y),
                 sep_constant=γ,
                 is_list = False,
                 factor=1,
                 check=True,
                 show_log=True,
                 show_eqs=True,
                 initial_pde=None)
show(eqs)

initialized
pre_separate test: passed!
partial_separate test: passed!
factor 1 eliminated
eliminate_common_factor test: passed!
common factor 1 ignored
partial_common_factor test: passed!
division terminate
initialized
pre_separate test: passed!
is_trivially_separable
canonicalize_de for X(x): passed!
initial_de: -γ*X(x) + diff(X(x), x, x) == 0
initial_de: -γ*X(x) + diff(X(x), x, x) == 0
new_de*coeff_2: -γ*X(x) + diff(X(x), x, x) == 0
canonicalize_de for Y(y): passed!
initial_de: γ*Y(y) + diff(Y(y), y, y) == 0
initial_de: γ*Y(y) + diff(Y(y), y, y) == 0
new_de*coeff_2: γ*Y(y) + diff(Y(y), y, y) == 0
separate_equations test 1: passed!
check_mix_diff test: passed!
separate_equations test 2: passed!


\(\large \color{green}{\text{Equations separated}}\)

\(\text{Parameters:} \left(x, γ\right) \)

\(\large -γ X\left(x\right) + \frac{\partial^2\,X}{\partial x ^ 2} = 0 \)

\(\text{Parameters:} \left(y, γ\right) \)

\(\large γ Y\left(y\right) + \frac{\partial^2\,Y}{\partial y ^ 2} = 0 \)

[-γ*X(x) + diff(X(x), x, x) == 0, γ*Y(y) + diff(Y(y), y, y) == 0]

### Declaração de variáveis e equações

In [8]:
X,Y,V = function('X')(x), function('Y')(y), function('V')(x)
var('k,i,m,h')

(k, i, m, h)

In [9]:
eq_hc = diff(F, x, 2) + diff(F, y ,2) + k**2 * F == 0 # equac¸ao de Helmholtz em coordenadas cartesianas
eq_hp = diff(x*diff(F, x), x)/x + diff(F, y, 2)/x**2 + k**2 * F == 0 # equac¸ao de Helmholtz em coordenadas polares

eq_laplace = diff(F, x, 2) + diff(F, y, 2) == 0 # equação de laplace potencial eletrostatico 
eq_calor = diff(F, y) == k*diff(F, x, 2) # eq. do calor
eq_onda = diff(F, y, 2) == k**2 * diff(F, x, 2) # eq. de onda

eq_sch = i*h*diff(F, y) == -h**2 * diff(F, x, 2)/(2*m) + V*F # Equação de Schrödinger dependente do tempo):

equations = [eq_hc, eq_hp, eq_laplace, eq_calor, eq_onda, eq_sch]

### Testes

In [27]:
for eqt in equations:
    show(eqt)
    separate_pde(pde=eqt.substitute_function(F==X*Y),
         funcs=(X,Y),
         sep_variables=(x,y),
         sep_constant=γ,
         is_list = False,
         factor=1,
         check=False,
         show_log=False,
         show_eqs=True,
         initial_pde=None)
    print('='*100)

k^2*F(x, y) + diff(F(x, y), x, x) + diff(F(x, y), y, y) == 0

division terminate


\(\large \color{green}{\text{Equations separated}}\)

\(\text{Parameters:} \left(x, γ\right) \)

\(\large -γ X\left(x\right) + \frac{\partial^2\,X}{\partial x ^ 2} = 0 \)

...

\(\large \color{green}{\text{Equations separated}}\)

\(\text{Parameters:} \left(h, i, m, x, γ\right) \)

\(\large -\frac{{\left(h^{3} i γ + 2 \, V\left(x\right)\right)} m X\left(x\right)}{h^{2}} + \frac{\partial^2\,X}{\partial x ^ 2} = 0 \)

\(\text{Parameters:} \left(h, y, γ\right) \)

\(\large h^{2} γ Y\left(y\right) + 2 \, \frac{\partial\,Y}{\partial y} = 0 \)

### Declarando variaveis e funções

In [11]:
var('M', domain='positive')
Scw.<t,r,θ,ϕ> = manifolds.Kerr(m=M,a=0)
chart = Scw.default_chart()
g = Scw.metric()

In [12]:
PR.<t,r,χ,ϕ> = Scw.chart(r't r:(0,oo) χ:[-1,1]:\chi ϕ:[-pi,pi]:periodic:\phi')
SC_to_PR = PR.transition_map(chart, [t,r,arccos(χ),ϕ])
Scw.set_default_chart(PR)
Scw.set_default_frame(PR.frame())

In [13]:
var('m μ λ', domain='positive') # μ = M / ħ, λ = sep_constant
var('ω')
R, S = function('R'),function('S')
Φ = Scw.scalar_field(exp(-i*ω*t+i*m*ϕ)*R(r)*S(χ),name='Φ',latex_name=r'\Phi')

###  Klein-gordon no espaço-tempo de Schwarzschild 

In [14]:
KG = -((r^3*χ^2*R(r) - r^3*R(r))*ω^2*S(χ) - ((2*M*r^2 - r^3)*μ^2*R(r) - ((2*M*r^2 - r^3)*μ^2*R(r) + 2*(2*M^2 - 3*M*r + r^2)*diff(R(r), r) + (4*M^2*r - 4*M*r^2 + r^3)*diff(R(r), r, r))*χ^2 + (2*M*m^2 - m^2*r)*R(r) + 2*(2*M^2 - 3*M*r + r^2)*diff(R(r), r) + (4*M^2*r - 4*M*r^2 + r^3)*diff(R(r), r, r))*S(χ) + 2*((2*M - r)*χ^3*R(r) - (2*M - r)*χ*R(r))*diff(S(χ), χ) + ((2*M - r)*χ^4*R(r) - 2*(2*M - r)*χ^2*R(r) + (2*M - r)*R(r))*diff(S(χ), χ, χ))/(((2*M*r^2 - r^3)*χ^2*R(r) - (2*M*r^2 - r^3)*R(r))*S(χ))
show(KG)

-((r^3*χ^2*R(r) - r^3*R(r))*ω^2*S(χ) - ((2*M*r^2 - r^3)*μ^2*R(r) - ((2*M*r^2 - r^3)*μ^2*R(r) + 2*(2*M^2 - 3*M*r + r^2)*diff(R(r), r) + (4*M^2*r - 4*M*r^2 + r^3)*diff(R(r), r, r))*χ^2 + (2*M*m^2 - m^2*r)*R(r) + 2*(2*M^2 - 3*M*r + r^2)*diff(R(r), r) + (4*M^2*r - 4*M*r^2 + r^3)*diff(R(r), r, r))*S(χ) + 2*((2*M - r)*χ^3*R(r) - (2*M - r)*χ*R(r))*diff(S(χ), χ) + ((2*M - r)*χ^4*R(r) - 2*(2*M - r)*χ^2*R(r) + (2*M - r)*R(r))*diff(S(χ), χ, χ))/(((2*M*r^2 - r^3)*χ^2*R(r) - (2*M*r^2 - r^3)*R(r))*S(χ))

In [28]:
eqt = KG
eqs = separate_pde(pde=eqt,
                 funcs=(R(r),S(χ)),
                 sep_variables=(r,χ),
                 sep_constant=γ,
                 is_list = False,
                 factor=1,
                 check=False,
                 show_log=False,
                 show_eqs=True,
                 initial_pde=None)
show(eqs)

\(\large \color{green}{\text{Equations separated}}\)

\(\text{Parameters:} \left(M, r, γ, μ, ω\right) \)

\(\large \frac{2 \, {\left(M - r\right)} \frac{\partial\,R}{\partial r}}{{\left(2 \, M - r\right)} r} + \frac{{\left(2 \, M r^{2} μ^{2} - r^{3} μ^{2} + r^{3} ω^{2} + 2 \, M γ - r γ\right)} R\left(r\right)}{{\left(2 \, M - r\right)}^{2} r} + \frac{\partial^2\,R}{\partial r ^ 2} = 0 \)

\(\text{Parameters:} \left(m, γ, {\chi}\right) \)

\(\large \frac{2 \, {\chi} \frac{\partial\,S}{\partial {\chi}}}{{\left({\chi} + 1\right)} {\left({\chi} - 1\right)}} - \frac{{\left(γ {\chi}^{2} + m^{2} - γ\right)} S\left({\chi}\right)}{{\left({\chi} + 1\right)}^{2} {\left({\chi} - 1\right)}^{2}} + \frac{\partial^2\,S}{\partial {\chi} ^ 2} = 0 \)

[2*(M - r)*diff(R(r), r)/((2*M - r)*r) + (2*M*r^2*μ^2 - r^3*μ^2 + r^3*ω^2 + 2*M*γ - r*γ)*R(r)/((2*M - r)^2*r) + diff(R(r), r, r) == 0,
 2*χ*diff(S(χ), χ)/((χ + 1)*(χ - 1)) - (γ*χ^2 + m^2 - γ)*S(χ)/((χ + 1)^2*(χ - 1)^2) + diff(S(χ), χ, χ) == 0]

## Próximos passos

* Permitir que o algoritmo possa lidar com decomposições de funções de muitas variáveis, por exemplo: $$F(r,\phi, \theta) = R(r)S(\phi,\theta)$$.
* Aumentar a base de testes a fim de investigar potenciais limitações do algoritmo e implementação.

#### Proxima equação

$$ \displaystyle \frac{{\left(-4 i \, a m r ω R\left(r\right) \sin\left({\theta}\right)^{2} \frac{\partial}{\partial {\phi}}S\left({\theta}, {\phi}\right) - {\left(a^{2} - 2 \, m r + r^{2}\right)} R\left(r\right) \cos\left({\theta}\right) \sin\left({\theta}\right) \frac{\partial}{\partial {\theta}}S\left({\theta}, {\phi}\right) - {\left(a^{2} - 2 \, m r + r^{2}\right)} R\left(r\right) \sin\left({\theta}\right)^{2} \frac{\partial^{2}}{(\partial {\theta})^{2}}S\left({\theta}, {\phi}\right) + {\left({\left(a^{4} - 2 \, a^{2} m r + a^{2} r^{2}\right)} R\left(r\right) \sin\left({\theta}\right)^{4} - {\left(a^{4} + 2 \, a^{2} r^{2} + r^{4}\right)} R\left(r\right) \sin\left({\theta}\right)^{2}\right)} ω^{2} S\left({\theta}, {\phi}\right) + {\left(2 \, {\left(a^{2} m + 3 \, m r^{2} - r^{3} - {\left(a^{2} + 2 \, m^{2}\right)} r\right)} \frac{\partial}{\partial r}R\left(r\right) - {\left(a^{4} - 4 \, a^{2} m r - 4 \, m r^{3} + r^{4} + 2 \, {\left(a^{2} + 2 \, m^{2}\right)} r^{2}\right)} \frac{\partial^{2}}{(\partial r)^{2}}R\left(r\right)\right)} S\left({\theta}, {\phi}\right) \sin\left({\theta}\right)^{2} + {\left(a^{2} R\left(r\right) \sin\left({\theta}\right)^{2} - {\left(a^{2} - 2 \, m r + r^{2}\right)} R\left(r\right)\right)} \frac{\partial^{2}}{(\partial {\phi})^{2}}S\left({\theta}, {\phi}\right)\right)} e^{\left(-i \, t ω\right)}}{{\left(a^{4} - 2 \, a^{2} m r + a^{2} r^{2}\right)} \cos\left({\theta}\right)^{4} - a^{2} r^{2} + 2 \, m r^{3} - r^{4} - {\left(a^{4} - 2 \, a^{2} m r + 2 \, m r^{3} - r^{4}\right)} \cos\left({\theta}\right)^{2}} = 0 $$